<a href="https://colab.research.google.com/github/LeonardoSer/Univ/blob/main/NetOpt/jupyter_notebooks/uncapacitatedLotSizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install -i https://pypi.gurobi.com gurobipy

import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gb
import random

%matplotlib inline

Looking in indexes: https://pypi.gurobi.com


In [ ]:
model = gb.Model('uncapacitated lot sizing')

T = [i for i in range(1, 11)]
fix_cost = {i:random.randint(0, 40) for i in T}
unit_cost = {i: random.randint(0,10) for i in T}
unit_storage_cost = {i:random.randint(0,5) for i in T}
demand = {i:random.randint(0, 100) for i in T}

In [ ]:
print(T, "\n", fix_cost, "\n", unit_cost, "\n", unit_storage_cost, "\n", demand)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
 {1: 28, 2: 13, 3: 11, 4: 29, 5: 17, 6: 6, 7: 31, 8: 19, 9: 19, 10: 14} 
 {1: 7, 2: 3, 3: 6, 4: 8, 5: 1, 6: 9, 7: 8, 8: 7, 9: 5, 10: 2} 
 {1: 1, 2: 5, 3: 0, 4: 4, 5: 1, 6: 2, 7: 4, 8: 0, 9: 0, 10: 3} 
 {1: 77, 2: 77, 3: 3, 4: 27, 5: 89, 6: 0, 7: 98, 8: 25, 9: 82, 10: 87}


In [ ]:
x = model.addVars(T, vtype=gb.GRB.CONTINUOUS, name='x')
y = model.addVars(T, vtype=gb.GRB.BINARY, name='y')
s = model.addVars(T, vtype=gb.GRB.CONTINUOUS, name='s')
model.update()
model.write("uncapacitated_lot_sizing.lp")

In [ ]:
model.setObjective(x.prod(unit_cost) + y.prod(fix_cost) + s.prod(unit_storage_cost))
model.update()
model.write("uncapacitated_lot_sizing.lp")

In [ ]:
model.addConstrs(((s[t-1]+x[t] == demand[t] + s[t]) for t in range(2, len(T))), name="balance")
model.update()
model.write("uncapacitated_lot_sizing.lp")

In [ ]:
bigM = 10000
model.addConstrs(((x[t] <= bigM*y[t]) for t in T), name="setup")
model.update()
model.write("uncapacitated_lot_sizing.lp")

In [ ]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 18 rows, 30 columns and 44 nonzeros
Model fingerprint: 0x35e9d3ab
Variable types: 20 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 10 rows and 18 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 20 nonzeros
Variable types: 7 continuous, 5 integer (5 binary)

Root relaxation: objective 1.363439e+03, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1363.43925    0    1          - 1363.43925      -     -    0s
H    0     0                    1378.0000000 1363.43925  1.06%     -    0s

Explored 1 nodes (6 simplex iterations

In [ ]:
print('Obj: %g' % model.objVal)

print ()

for v in model.getVars():
    if v.x > 0.1:
        print ('%s=%g' % (v.varName, v.x), end = ' ')

Obj: 1378

x[5]=187 x[8]=25 x[9]=82 y[5]=1 y[8]=1 y[9]=1 s[1]=107 s[2]=30 s[3]=27 s[5]=98 s[6]=98 